# 言語処理100本ノック 2020 第３章

Reference: https://nlp100.github.io/ja/ch03.html

In [5]:
!wget https://nlp100.github.io/data/jawiki-country.json.gz

In [27]:
import json, gzip
import numpy as np

## 20. JSONデータの読み込み

Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [25]:
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        if tmp["title"] == "イギリス": break
tmp["text"][:200]

'{{redirect|UK}}\n{{redirect|英国|春秋時代の諸侯国|英 (春秋)}}\n{{Otheruses|ヨーロッパの国|長崎県・熊本県の郷土料理|いぎりす}}\n{{基礎情報 国\n|略名  =イギリス\n|日本語国名 = グレートブリテン及び北アイルランド連合王国\n|公式国名 = {{lang|en|United Kingdom of Great Britain and Norther'

## 21. カテゴリ名を含む行を抽出

記事中でカテゴリ名を宣言している行を抽出せよ．

In [51]:
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    cat_lines = []
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        lines = tmp["text"].split("\n")
        cat_lines += [line for line in lines if "Category:" in line]
cat_lines[:5]

['[[Category:エジプト|*]]',
 '[[Category:共和国]]',
 '[[Category:軍事政権]]',
 '[[Category:フランコフォニー加盟国]]',
 '{{Main|オーストリア料理|Category:オーストリアの食文化}}']

## 22. カテゴリ名の抽出

記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [102]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    categories = []
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        lines = tmp["text"].split("\n")
        categories += [re.split("\\]|\\)|\\|", line.strip("[](){}").split("Category:")[1])[0] for line in lines if "Category:" in line]
np.unique(categories)[:5]

array(['1704年設立', '1801年に成立した国家・領域', '1804年に成立した国家・領域', '1886年に成立した国家・領域',
       '1922年に成立した国家・領域'], dtype='<U32')

## 23. セクション構造

記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [101]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    sections = []
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        lines = tmp["text"].split("\n")
        for l in lines:
            if "|section=" in l:
                s_s = l.split("|")
                s_title = s_s[0].strip("{")
                s_level = re.search("section=[0-9]*", l).group().split("=")[1]
                sections.append((s_title,s_level))
sections[:5]

[('独自研究', '1'), ('出典の明記', '1'), ('独自研究', '1'), ('出典の明記', '1'), ('出典の明記', '1')]

## 24. ファイル参照の抽出

記事から参照されているメディアファイルをすべて抜き出せ

In [107]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    files = []
    for i,j in enumerate(fi):
        tmp = json.loads(j)
        lines = tmp["text"].split("\n")
        files += [re.search("ファイル:.*", l).group().split(":")[1].split("|")[0].split("]")[0] for l in lines if "ファイル:" in l]
files[:5]

['Coat_of_arms_of_Egypt.svg',
 'All Gizah Pyramids.jpg',
 'Egyptiska hieroglyfer, Nordisk familjebok.png',
 'ModernEgypt, Muhammad Ali by Auguste Couder, BAP 17996.jpg',
 'Gamal Nasser.jpg']

## 25. テンプレートの抽出

記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [191]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    result = dict()
    for i,j in enumerate(fi):
        d = dict()
        tmp = json.loads(j)
        ll = re.search("{{基礎情報 (.|\n)+\n}}", tmp["text"])
        if ll is None: continue
        ll = ll.group().split("\n|")
        for l in ll[1:]: 
            if not "=" in l: continue
            d[l.split("=")[0].strip(" ")] = l.split("=")[1].strip(" ")
        result[tmp["title"]] = d
result.keys()


dict_keys(['エジプト', 'オーストリア', 'インドネシア', 'イラク', 'イラン', 'チュニジア', 'トルコ', 'ロシア', 'コスタリカ', 'ラオス', '中華人民共和国', 'インド', 'パラグアイ', 'アルゼンチン', 'ウルグアイ', 'チリ', 'ボリビア', 'ペルー', 'メキシコ', 'スイス', 'イタリア', 'バチカン', 'エクアドル', 'イスラエル', 'ギリシャ', 'サウジアラビア', 'クウェート', 'スリランカ', 'アフガニスタン', 'ポルトガル', 'ナイジェリア', 'ヨルダン', 'パキスタン', 'レバノン', 'イエメン', 'オマーン', 'ネパール', 'クロアチア', 'スロベニア', 'アイルランド', 'ポーランド', 'バーレーン', 'ニュージーランド', 'ジャマイカ', 'アルメニア', 'ノルウェー', 'トルクメニスタン', 'コロンビア', 'チェコ', 'ブータン', 'エストニア', 'モロッコ', 'ソマリア', 'スロバキア', 'アラブ首長国連邦', 'キプロス', 'バングラデシュ', 'ブルネイ', 'アルジェリア', 'アンゴラ', 'ウガンダ', 'ガーナ', 'カメルーン', 'ギニア', 'ケニア', 'マリ共和国', 'スーダン', 'モザンビーク', 'マカオ', 'ニジェール', 'ルーマニア', 'ハンガリー', 'ウクライナ', '北マケドニア', 'コンゴ民主共和国', 'モーリタニア', 'カザフスタン', 'ジブチ', 'コモロ', '東ティモール', 'アルバニア', 'ボスニア・ヘルツェゴビナ', 'ベラルーシ', 'タンザニア', 'フィジー', 'シリア', 'ミクロネシア連邦', 'ルクセンブルク', 'マルタ', 'リヒテンシュタイン', 'モナコ', 'キューバ', 'モーリシャス', 'ウズベキスタン', 'モンゴル国', 'コンゴ共和国', 'タジキスタン', 'マラウイ', 'ジンバブエ', 'サンマリノ', 'ガイアナ', 'ニカラグア', 'ジョージア (国)', 'レソト', 'リトアニア', 'モルドバ', 'アゼルバイジャン', 'トンガ', 'アンドラ', 'ベネズエラ', 'カーボベルデ'

In [192]:
result["インドネシア"]

{'GDP/人': '13,234<ref name',
 'GDP値': '3兆8,706億<ref name',
 'GDP値MER': '1兆225億<ref name',
 'GDP値元': '1京4,825兆9,440億<ref name',
 'GDP統計年': '2018',
 'GDP統計年MER': '2018',
 'GDP統計年元': '2018',
 'GDP順位': '7',
 'GDP順位MER': '16',
 'ISO 3166-1': 'ID / IDN',
 'bars': '\n{{bar percent|[[イスラム教]]|Green|87.2}}\n{{bar percent|[[プロテスタント]]|violet|7}}\n{{bar percent|[[カトリック]]|purple|2.9}}\n{{bar percent|[[ヒンドゥー教]]|Orange|1.6}}\n{{bar percent|[[仏教]]|Gold|0.72}}\n{{bar percent|[[儒教]]|Blue|0.05}}\n{{bar percent|その他|grey|0.5}}\n}}',
 'ccTLD': '[[.id]]',
 'float': 'right',
 'left1': '宗教',
 'right1': '割合(%)',
 'title': 'インドネシアの宗教<ref>{{cite web |url',
 'titlebar': '#ddd',
 '人口値': '264,162,000<ref name',
 '人口大きさ': '1 E8',
 '人口密度値': '138',
 '人口統計年': '2018',
 '人口順位': '4',
 '位置画像': 'Indonesia (orthographic projection).svg',
 '元首等氏名': '[[ジョコ・ウィドド]]',
 '元首等肩書': '[[インドネシアの大統領一覧|大統領]]',
 '公式国名': "'''{{Lang|id|Republik Indonesia}}'''",
 '公用語': '[[インドネシア語]]',
 '国旗画像': 'Flag of Indonesia.svg',
 '国歌': "[[インドネシア・ラヤ|{{lang

## 26. 強調マークアップの除去

25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）

In [195]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    result = dict()
    for i,j in enumerate(fi):
        d = dict()
        tmp = json.loads(j)
        ll = re.search("{{基礎情報 (.|\n)+\n}}", tmp["text"])
        if ll is None: continue
        ll = ll.group().split("\n|")
        for l in ll[1:]: 
            if not "=" in l: continue
            d[l.split("=")[0].strip(" ")] = re.sub("(''''')|(''')|('')", "", l.split("=")[1].strip(" "))
        result[tmp["title"]] = d
result["インドネシア"]["公式国名"]

'{{Lang|id|Republik Indonesia}}'

## 27. 内部リンクの除去

26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [271]:
import re
with gzip.open("jawiki-country.json.gz", "rt", encoding="utf-8") as fi:
    result = dict()
    for i,j in enumerate(fi):
        d = dict()
        tmp = json.loads(j)
        ll = re.search("{{基礎情報 (.|\n)+\n}}", tmp["text"])
        if ll is None: continue
        ll = ll.group().split("\n|")
        for l in ll[1:]: 
            if not "=" in l: continue
            cont = l.split("=")[1].strip(" ")
            cont = re.sub("(''''')|(''')|('')", "", cont) # 強調マークアップを除去
            # 内部リンクの除去
            for i in re.findall("\[{2}[\w|]+\]{2}", cont): 
                cont = cont.replace(i, i.strip("[]").split("|")[0])
            d[l.split("=")[0].strip(" ")] = cont
        result[tmp["title"]] = d
result["インドネシア"]["注記"]

'\n}}\nインドネシア共和国（インドネシアきょうわこく、{{Lang-id|Republik Indonesia}}）、通称インドネシアは、東南アジア南部に位置する共和制国家。首都はジャワ島に位置するジャカルタ。\n\n5,110kmと東西に非常に長く、また世界最多の島嶼を抱える国家（島国）である。赤道にまたがる1万3,466もの大小の島により構成される<ref name'

'ア]'